In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

In [ ]:
import sys
sys.path.append('/content/gaussian-splatting')

In [ ]:
#
# Code are Adapted From ----
# Copyright (C) 2023, Inria
# GRAPHDECO research group, https://team.inria.fr/graphdeco
# All rights reserved.
#

# System imports
import os
import sys
import uuid

# Torch things
import torch
import numpy as np
from random import randint
from torch import nn
from tqdm import tqdm

# Utils function from the original repo
from scene import Scene
from utils.loss_utils import l1_loss, ssim
from gaussian_renderer import render, network_gui
from utils.general_utils import safe_state
from utils.image_utils import psnr
from utils.general_utils import inverse_sigmoid, get_expon_lr_func, build_rotation
from utils.system_utils import mkdir_p
from plyfile import PlyData, PlyElement
from utils.sh_utils import RGB2SH
from simple_knn._C import distCUDA2
from utils.graphics_utils import BasicPointCloud
from utils.general_utils import strip_symmetric, build_scaling_rotation

In [ ]:
def build_covariance_from_scaling_rotation(scaling, scaling_modifier, rotation):
    L = build_scaling_rotation(scaling_modifier * scaling, rotation)
    actual_covariance = L @ L.transpose(1, 2)
    symm = strip_symmetric(actual_covariance)
    return symm

class GaussianModel(nn.Module):

    def __init__(self, sh_degree : int):
        super().__init__()
        self.active_sh_degree = 0
        self.max_sh_degree = sh_degree
        self._xyz = torch.empty(0)
        self._features_dc = torch.empty(0)
        self._features_rest = torch.empty(0)
        self._scaling = torch.empty(0)
        self._rotation = torch.empty(0)
        self._opacity = torch.empty(0)
        self.max_radii2D = torch.empty(0)
        self.xyz_gradient_accum = torch.empty(0)
        self.denom = torch.empty(0)
        self.optimizer = None
        self.percent_dense = 0
        self.spatial_lr_scale = 0

    def forward(self, viewpoint_cam, gaussians, pipe, bg):
        render_pkg = render(viewpoint_cam, gaussians, pipe, bg)
        image, viewspace_point_tensor, visibility_filter, radii = render_pkg["render"], render_pkg["viewspace_points"], render_pkg["visibility_filter"], render_pkg["radii"]
        return image, viewspace_point_tensor, visibility_filter, radii

    def loss(self, image, gt_image, opt):
        Ll1 = l1_loss(image, gt_image)
        loss = (1.0 - opt.lambda_dssim) * Ll1 + opt.lambda_dssim * (1.0 - ssim(image, gt_image))
        return loss

    def get_learning_params(self):
        return {
            "xyz": self._xyz,
            "features_dc": self._features_dc,
            "features_rest": self._features_rest,
            "opacity": self._opacity,
            "scaling": self._scaling,
            "rotation": self._rotation
        }

    @property
    def get_scaling(self):
        return torch.exp(self._scaling)

    @property
    def get_rotation(self):
        return torch.nn.functional.normalize(self._rotation)

    @property
    def get_xyz(self):
        return self._xyz

    @property
    def get_features(self):
        features_dc = self._features_dc
        features_rest = self._features_rest
        return torch.cat((features_dc, features_rest), dim=1)

    @property
    def get_opacity(self):
        return torch.sigmoid(self._opacity)

    def get_covariance(self, scaling_modifier = 1):
        return build_covariance_from_scaling_rotation(self.get_scaling, scaling_modifier, self._rotation)

    def oneupSHdegree(self):
        if self.active_sh_degree < self.max_sh_degree:
            self.active_sh_degree += 1

    def reset_opacity(self, optimizer):
        opacities_new = inverse_sigmoid(torch.min(self.get_opacity, torch.ones_like(self.get_opacity)*0.01))
        optimizable_tensors = self.replace_tensor_to_optimizer(opacities_new, "opacity", optimizer)
        self._opacity = optimizable_tensors["opacity"]

    def replace_tensor_to_optimizer(self, tensor, name, optimizer):
        # update the training params accordinly
        optimizable_tensors = {}
        for group in optimizer.param_groups:
            if group["name"] == name:
                stored_state = optimizer.state.get(group['params'][0], None)
                stored_state["exp_avg"] = torch.zeros_like(tensor)
                stored_state["exp_avg_sq"] = torch.zeros_like(tensor)

                del optimizer.state[group['params'][0]]
                group["params"][0] = nn.Parameter(tensor.requires_grad_(True))
                optimizer.state[group['params'][0]] = stored_state

                optimizable_tensors[group["name"]] = group["params"][0]
        return optimizable_tensors

    def _prune_optimizer(self, mask, optimizer):
        # we also need to remove the points on the optimizer so we dont make unnesscary computations
        optimizable_tensors = {}
        for group in optimizer.param_groups:
            stored_state = optimizer.state.get(group['params'][0], None)
            if stored_state is not None:
                stored_state["exp_avg"] = stored_state["exp_avg"][mask]
                stored_state["exp_avg_sq"] = stored_state["exp_avg_sq"][mask]

                del optimizer.state[group['params'][0]]
                group["params"][0] = nn.Parameter((group["params"][0][mask].requires_grad_(True)))
                optimizer.state[group['params'][0]] = stored_state

                optimizable_tensors[group["name"]] = group["params"][0]
            else:
                group["params"][0] = nn.Parameter(group["params"][0][mask].requires_grad_(True))
                optimizable_tensors[group["name"]] = group["params"][0]
        return optimizable_tensors

    def prune_points(self, mask, optimizer):
        valid_points_mask = ~mask

        # remove points that does not satisfy the mask
        optimizable_tensors = self._prune_optimizer(valid_points_mask, optimizer)

        self._xyz = optimizable_tensors["xyz"]
        self._features_dc = optimizable_tensors["f_dc"]
        self._features_rest = optimizable_tensors["f_rest"]
        self._opacity = optimizable_tensors["opacity"]
        self._scaling = optimizable_tensors["scaling"]
        self._rotation = optimizable_tensors["rotation"]

        # retain only gradient of points not pruned

        self.xyz_gradient_accum = self.xyz_gradient_accum[valid_points_mask]

        self.denom = self.denom[valid_points_mask]
        self.max_radii2D = self.max_radii2D[valid_points_mask]

    def cat_tensors_to_optimizer(self, tensors_dict, optimizer):
        optimizable_tensors = {}
        for group in optimizer.param_groups:
            assert len(group["params"]) == 1
            extension_tensor = tensors_dict[group["name"]]
            stored_state = optimizer.state.get(group['params'][0], None)
            if stored_state is not None:

                stored_state["exp_avg"] = torch.cat((stored_state["exp_avg"], torch.zeros_like(extension_tensor)), dim=0)
                stored_state["exp_avg_sq"] = torch.cat((stored_state["exp_avg_sq"], torch.zeros_like(extension_tensor)), dim=0)

                del optimizer.state[group['params'][0]]
                group["params"][0] = nn.Parameter(torch.cat((group["params"][0], extension_tensor), dim=0).requires_grad_(True))
                optimizer.state[group['params'][0]] = stored_state

                optimizable_tensors[group["name"]] = group["params"][0]
            else:
                group["params"][0] = nn.Parameter(torch.cat((group["params"][0], extension_tensor), dim=0).requires_grad_(True))
                optimizable_tensors[group["name"]] = group["params"][0]

        return optimizable_tensors

    def densification_postfix(self, new_xyz, new_features_dc, new_features_rest, new_opacities, new_scaling, new_rotation, optimizer):
        d = {
            "xyz": new_xyz,
            "f_dc": new_features_dc,
            "f_rest": new_features_rest,
            "opacity": new_opacities,
            "scaling" : new_scaling,
            "rotation" : new_rotation
        }

        # add new points to be optimized as well as part of the model
        optimizable_tensors = self.cat_tensors_to_optimizer(d, optimizer)

        # update each class fields accordngly
        self._xyz = optimizable_tensors["xyz"]
        self._features_dc = optimizable_tensors["f_dc"]
        self._features_rest = optimizable_tensors["f_rest"]
        self._opacity = optimizable_tensors["opacity"]
        self._scaling = optimizable_tensors["scaling"]
        self._rotation = optimizable_tensors["rotation"]

        self.xyz_gradient_accum = torch.zeros((self.get_xyz.shape[0], 1), device="cuda")
        self.denom = torch.zeros((self.get_xyz.shape[0], 1), device="cuda")
        self.max_radii2D = torch.zeros((self.get_xyz.shape[0]), device="cuda")

    def densify_and_split(self, grads, grad_threshold, scene_extent, optimizer, N=2):
        n_init_points = self.get_xyz.shape[0]
        # Extract points that satisfy the gradient condition
        padded_grad = torch.zeros((n_init_points), device="cuda")
        padded_grad[:grads.shape[0]] = grads.squeeze()

        # Select gaussian to split if the gradient is more than the threshold

        mask = torch.where(padded_grad >= grad_threshold, True, False)
        mask = torch.logical_and(mask, torch.max(self.get_scaling, dim=1).values > self.percent_dense*scene_extent)

        # create new features set for each splited points

        stds = self.get_scaling[mask].repeat(N,1)
        means =torch.zeros((stds.size(0), 3),device="cuda")
        samples = torch.normal(mean=means, std=stds)
        rots = build_rotation(self._rotation[mask]).repeat(N,1,1)

        new_xyz = torch.bmm(rots, samples.unsqueeze(-1)).squeeze(-1) + self.get_xyz[mask].repeat(N, 1)
        new_scaling = torch.log(self.get_scaling[mask].repeat(N,1) / (0.8*N))
        new_rotation = self._rotation[mask].repeat(N,1)
        new_features_dc = self._features_dc[mask].repeat(N,1,1)
        new_features_rest = self._features_rest[mask].repeat(N,1,1)
        new_opacity = self._opacity[mask].repeat(N,1)

        # Concat new points to the avaiable points
        self.densification_postfix(new_xyz, new_features_dc, new_features_rest, new_opacity, new_scaling, new_rotation, optimizer)

        # Also prune useless points
        zeroes = torch.zeros(N * mask.sum(), device="cuda", dtype=bool)
        prune_filter = torch.cat((mask, zeroes))
        self.prune_points(prune_filter, optimizer)

    def densify_and_clone(self, grads, grad_threshold, scene_extent, optimizer):
        # Same as split
        mask = torch.where(torch.norm(grads, dim=-1) >= grad_threshold, True, False)
        mask = torch.logical_and(mask, torch.max(self.get_scaling, dim=1).values <= self.percent_dense * scene_extent)

        new_xyz = self._xyz[mask]
        new_features_dc = self._features_dc[mask]
        new_features_rest = self._features_rest[mask]
        new_opacities = self._opacity[mask]
        new_scaling = self._scaling[mask]
        new_rotation = self._rotation[mask]

        # Update learning parameters
        self.densification_postfix(new_xyz, new_features_dc, new_features_rest, new_opacities, new_scaling, new_rotation, optimizer)

    def densify_and_prune(self, max_grad, min_opacity, extent, max_screen_size, optimizer):
        grads = self.xyz_gradient_accum / self.denom
        grads[grads.isnan()] = 0.0

        self.densify_and_clone(grads, max_grad, extent, optimizer)
        self.densify_and_split(grads, max_grad, extent, optimizer)

        # remove useless almost no color gaussians

        prune_mask = (self.get_opacity < min_opacity).squeeze()
        if max_screen_size:
            big_points_vs = self.max_radii2D > max_screen_size
            big_points_ws = self.get_scaling.max(dim=1).values > 0.1 * extent
            prune_mask = torch.logical_or(torch.logical_or(prune_mask, big_points_vs), big_points_ws)
        self.prune_points(prune_mask, optimizer)

        # remove unused stuff from VRAM

        torch.cuda.empty_cache()

    def add_densification_stats(self, viewspace_point_tensor, update_filter):
        self.xyz_gradient_accum[update_filter] += torch.norm(viewspace_point_tensor.grad[update_filter,:2], dim=-1, keepdim=True)
        self.denom[update_filter] += 1

    def capture(self, optimizer):
        return (
            self.active_sh_degree,
            self._xyz,
            self._features_dc,
            self._features_rest,
            self._scaling,
            self._rotation,
            self._opacity,
            self.max_radii2D,
            self.xyz_gradient_accum,
            self.denom,
            optimizer.state_dict(),
            self.spatial_lr_scale,
        )

    def create_from_pcd(self, pcd : BasicPointCloud, spatial_lr_scale : float):
        self.spatial_lr_scale = spatial_lr_scale
        fused_point_cloud = torch.tensor(np.asarray(pcd.points)).float().cuda()
        fused_color = RGB2SH(torch.tensor(np.asarray(pcd.colors)).float().cuda())
        features = torch.zeros((fused_color.shape[0], 3, (self.max_sh_degree + 1) ** 2)).float().cuda()
        features[:, :3, 0 ] = fused_color
        features[:, 3:, 1:] = 0.0

        dist2 = torch.clamp_min(distCUDA2(torch.from_numpy(np.asarray(pcd.points)).float().cuda()), 0.0000001)
        scales = torch.log(torch.sqrt(dist2))[...,None].repeat(1, 3)
        rots = torch.zeros((fused_point_cloud.shape[0], 4), device="cuda")
        rots[:, 0] = 1

        opacities = inverse_sigmoid(0.1 * torch.ones((fused_point_cloud.shape[0], 1), dtype=torch.float, device="cuda"))

        self._xyz = nn.Parameter(fused_point_cloud.requires_grad_(True))
        self._features_dc = nn.Parameter(features[:,:,0:1].transpose(1, 2).contiguous().requires_grad_(True))
        self._features_rest = nn.Parameter(features[:,:,1:].transpose(1, 2).contiguous().requires_grad_(True))
        self._scaling = nn.Parameter(scales.requires_grad_(True))
        self._rotation = nn.Parameter(rots.requires_grad_(True))
        self._opacity = nn.Parameter(opacities.requires_grad_(True))
        self.max_radii2D = torch.zeros((self.get_xyz.shape[0]), device="cuda")

    def construct_list_of_attributes(self):
        l = ['x', 'y', 'z', 'nx', 'ny', 'nz']
        # All channels except the 3 DC
        for i in range(self._features_dc.shape[1]*self._features_dc.shape[2]):
            l.append('f_dc_{}'.format(i))
        for i in range(self._features_rest.shape[1]*self._features_rest.shape[2]):
            l.append('f_rest_{}'.format(i))
        l.append('opacity')
        for i in range(self._scaling.shape[1]):
            l.append('scale_{}'.format(i))
        for i in range(self._rotation.shape[1]):
            l.append('rot_{}'.format(i))
        return l

    def save_ply(self, path):
        mkdir_p(os.path.dirname(path))

        xyz = self._xyz.detach().cpu().numpy()
        normals = np.zeros_like(xyz)
        f_dc = self._features_dc.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
        f_rest = self._features_rest.detach().transpose(1, 2).flatten(start_dim=1).contiguous().cpu().numpy()
        opacities = self._opacity.detach().cpu().numpy()
        scale = self._scaling.detach().cpu().numpy()
        rotation = self._rotation.detach().cpu().numpy()

        dtype_full = [(attribute, 'f4') for attribute in self.construct_list_of_attributes()]

        elements = np.empty(xyz.shape[0], dtype=dtype_full)
        attributes = np.concatenate((xyz, normals, f_dc, f_rest, opacities, scale, rotation), axis=1)
        elements[:] = list(map(tuple, attributes))
        el = PlyElement.describe(elements, 'vertex')
        PlyData([el]).write(path)

In [ ]:
class ModelParams:
    def __init__(self, sh_degree: int, source_path: str, model_path: str, images: str, resolution: int, white_background: bool, data_device: str, eval: bool):
        self.sh_degree = sh_degree
        self.source_path = source_path
        self.model_path = model_path
        self.images = images
        self.resolution = resolution
        self.white_background = white_background
        self.data_device = data_device
        self.eval = eval

    def __str__(self):
        return (f"ModelParams(sh_degree={self.sh_degree}, source_path={self.source_path}, "
                f"model_path={self.model_path}, images={self.images}, resolution={self.resolution}, "
                f"white_background={self.white_background}, data_device={self.data_device}, eval={self.eval})")

class PipelineParams:
    def __init__(self, convert_SHs_python: bool, compute_cov3D_python: bool, debug: bool):
        self.convert_SHs_python = convert_SHs_python
        self.compute_cov3D_python = compute_cov3D_python
        self.debug = debug

    def __str__(self):
        return (f"PipelineParams(convert_SHs_python={self.convert_SHs_python}, "
                f"compute_cov3D_python={self.compute_cov3D_python}, debug={self.debug})")

class OptimizationParams:
    def __init__(self, iterations, position_lr_init, position_lr_final, position_lr_delay_mult, position_lr_max_steps, feature_lr, opacity_lr, scaling_lr, rotation_lr, percent_dense, lambda_dssim, densification_interval, opacity_reset_interval, densify_from_iter, densify_until_iter, densify_grad_threshold, random_background):
        self.iterations = iterations
        self.position_lr_init = position_lr_init
        self.position_lr_final = position_lr_final
        self.position_lr_delay_mult = position_lr_delay_mult
        self.position_lr_max_steps = position_lr_max_steps
        self.feature_lr = feature_lr
        self.opacity_lr = opacity_lr
        self.scaling_lr = scaling_lr
        self.rotation_lr = rotation_lr
        self.percent_dense = percent_dense
        self.lambda_dssim = lambda_dssim
        self.densification_interval = densification_interval
        self.opacity_reset_interval = opacity_reset_interval
        self.densify_from_iter = densify_from_iter
        self.densify_until_iter = densify_until_iter
        self.densify_grad_threshold = densify_grad_threshold
        self.random_background = random_background

    def __str__(self):
        return (f"OptimizationParams(iterations={self.iterations}, position_lr_init={self.position_lr_init}, "
                f"position_lr_final={self.position_lr_final}, position_lr_delay_mult={self.position_lr_delay_mult}, "
                f"position_lr_max_steps={self.position_lr_max_steps}, feature_lr={self.feature_lr}, "
                f"opacity_lr={self.opacity_lr}, scaling_lr={self.scaling_lr}, rotation_lr={self.rotation_lr}, "
                f"percent_dense={self.percent_dense}, lambda_dssim={self.lambda_dssim}, "
                f"densification_interval={self.densification_interval}, opacity_reset_interval={self.opacity_reset_interval}, "
                f"densify_from_iter={self.densify_from_iter}, densify_until_iter={self.densify_until_iter}, "
                f"densify_grad_threshold={self.densify_grad_threshold}, random_background={self.random_background})")


In [ ]:


def update_learning_rate(
        iteration,
        optimizer,
        scheduler
    ):
    ''' Learning rate scheduling per step '''
    for param_group in optimizer.param_groups:
        if param_group["name"] == "xyz":
            lr = scheduler(iteration)
            param_group['lr'] = lr
            return lr

def training(
        dataset: ModelParams,
        opt: OptimizationParams,
        pipe: PipelineParams,
        testing_iterations,
        saving_iterations,
        debug_from
    ):
    first_iter = 0
    gaussians = GaussianModel(dataset.sh_degree)
    scene = Scene(dataset, gaussians)

    # initialize gaussians

    gaussians.percent_dense = opt.percent_dense
    gaussians.xyz_gradient_accum = torch.zeros((gaussians.get_xyz.shape[0], 1), device="cuda")
    gaussians.denom = torch.zeros((gaussians.get_xyz.shape[0], 1), device="cuda")

    params_dict = gaussians.get_learning_params()

    # initilize traning parameters

    l = [
        {'params': [params_dict["xyz"]], 'lr': opt.position_lr_init * gaussians.spatial_lr_scale, "name": "xyz"},
        {'params': [params_dict["features_dc"]], 'lr': opt.feature_lr, "name": "f_dc"},
        {'params': [params_dict["features_rest"]], 'lr': opt.feature_lr / 20.0, "name": "f_rest"},
        {'params': [params_dict["opacity"]], 'lr': opt.opacity_lr, "name": "opacity"},
        {'params': [params_dict["scaling"]], 'lr': opt.scaling_lr, "name": "scaling"},
        {'params': [params_dict["rotation"]], 'lr': opt.rotation_lr, "name": "rotation"}
    ]

    # initializer and picking our choice of optimizer with learning rate scheduling

    optimizer = torch.optim.Adam(l, lr=0.0, eps=1e-15)
    xyz_scheduler_args = get_expon_lr_func(lr_init=opt.position_lr_init*gaussians.spatial_lr_scale,
                                                    lr_final=opt.position_lr_final*gaussians.spatial_lr_scale,
                                                    lr_delay_mult=opt.position_lr_delay_mult,
                                                    max_steps=opt.position_lr_max_steps)

    bg_color = [1, 1, 1] if dataset.white_background else [0, 0, 0]
    background = torch.tensor(bg_color, dtype=torch.float32, device="cuda")

    viewpoint_stack = None
    ema_loss_for_log = 0.0
    progress_bar = tqdm(range(first_iter, opt.iterations), desc="Training progress")
    first_iter += 1

    for iteration in range(first_iter, opt.iterations + 1):

        # updat ethe learning rate every iteration according to the sceduler
        update_learning_rate(
            iteration,
            optimizer,
            xyz_scheduler_args
        )

        # Every 1000 its we increase the levels of SH up to a maximum degree
        if iteration % 1000 == 0:
            gaussians.oneupSHdegree()

        # Pick a random Camera (picture in the traning set)
        if not viewpoint_stack:
            viewpoint_stack = scene.getTrainCameras().copy()
        viewpoint_cam = viewpoint_stack.pop(randint(0, len(viewpoint_stack)-1))

        bg = torch.rand((3), device="cuda") if opt.random_background else background

        # feed forward pass

        image, viewspace_point_tensor, visibility_filter, radii = gaussians(viewpoint_cam, gaussians, pipe, bg)

        # actual image
        gt_image = viewpoint_cam.original_image.cuda()

        # loss calculation and back prop
        loss = gaussians.loss(image, gt_image, opt)
        loss.backward()

        with torch.no_grad():
            # Progress bar
            ema_loss_for_log = 0.4 * loss.item() + 0.6 * ema_loss_for_log
            if iteration % 10 == 0:
                progress_bar.set_postfix({"Loss": f"{ema_loss_for_log:.{7}f}"})
                progress_bar.update(10)
            if iteration == opt.iterations:
                progress_bar.close()

            # Log and save
            # training_report(tb_writer, iteration, Ll1, loss, l1_loss, iter_start.elapsed_time(iter_end), testing_iterations, scene, render, (pipe, background))
            if (iteration in saving_iterations):
                print("\n[ITER {}] Saving Gaussians".format(iteration))
                scene.save(iteration)

            # Densification
            if iteration < opt.densify_until_iter:
                # Keep track of max radii in image-space for pruning
                gaussians.max_radii2D[visibility_filter] = torch.max(gaussians.max_radii2D[visibility_filter], radii[visibility_filter])
                gaussians.add_densification_stats(viewspace_point_tensor, visibility_filter)

                # densify after some fixed iteration
                if iteration > opt.densify_from_iter and iteration % opt.densification_interval == 0:
                    size_threshold = 20 if iteration > opt.opacity_reset_interval else None
                    gaussians.densify_and_prune(opt.densify_grad_threshold, 0.005, scene.cameras_extent, size_threshold, optimizer)

                if iteration % opt.opacity_reset_interval == 0 or (dataset.white_background and iteration == opt.densify_from_iter):
                    gaussians.reset_opacity(optimizer)

            # Optimizer step
            if iteration < opt.iterations:
                optimizer.step()
                optimizer.zero_grad(set_to_none = True)

In [ ]:
# We connect to google drive here for our own data. Change this accordingly
source_path = "/content/drive/MyDrive/DL Project 2/1502-1"
model_path = "/content/drive/MyDrive/DL Project 2/1502-1"

optiimzation_params = OptimizationParams(
    iterations = 30_000,
    position_lr_init = 0.00016,
    position_lr_final = 0.0000016,
    position_lr_delay_mult = 0.01,
    position_lr_max_steps = 30_000,
    feature_lr = 0.0025,
    opacity_lr = 0.05,
    scaling_lr = 0.005,
    rotation_lr = 0.001,
    percent_dense = 0.01,
    lambda_dssim = 0.2,
    densification_interval = 100,
    opacity_reset_interval = 3000,
    densify_from_iter = 500,
    densify_until_iter = 15_000,
    densify_grad_threshold = 0.0002,
    random_background = False
)

model_params = ModelParams(
    sh_degree = 3,
    source_path = source_path,
    model_path = model_path,
    images = "images",
    resolution = -1,
    white_background = False,
    data_device = "cuda",
    eval = False
)

pipeline_params = PipelineParams(
    convert_SHs_python = False,
    compute_cov3D_python = False,
    debug = False
)

test_iterations = [7000, 30000]
save_iterations = [7000, 30000]
debug_from = -1

print(optiimzation_params)
print(model_params)
print(pipeline_params)

In [ ]:
torch.autograd.set_detect_anomaly(False)

training(
    model_params,
    optiimzation_params,
    pipeline_params,
    test_iterations,
    save_iterations,
    debug_from
)

# All done
print("\nTraining complete.")

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()